In [100]:
import io
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from textblob import TextBlob

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook', rc={'figure.figsize': (10, 6)}, font_scale=1.5)

In [80]:
stopwords = set(stopwords.words('english'))
stopwords.update(['http','https','www'])

In [50]:
input_data = 'clean_scraped_text/clean_scraped_text.txt'
data = []
with io.open(input_data,'r',encoding="utf-8") as f:
    for line in f:
        data.append(eval(line.strip('\n')))

In [58]:
df = pd.DataFrame()
for row in data:
    temp = pd.DataFrame.from_dict({(i,j): row[i][j] 
                           for i in row.keys() 
                           for j in row[i].keys()},
                       orient='index')
    df = df.append(temp)

In [61]:
df.reset_index(inplace=True)

In [63]:
df.head()

,level_0,level_1,text,entities
0,active-democracy.com,index.html,search search home about us active democracy ...,"[{u'Search ': u'PERSON'}, {u'Us': u'GP..."
1,addr.ws,the-nation-of-islam-lexington-ky-study-group--...,just addr lexington the nation of islam lexin...,"[{u' ': u'GPE'}, {u'Lexington': u'GP..."
2,alternative-right.blogspot.com,index.html,friday 10 november 2017 unscrolling the narra...,[{u' ...
3,americaneagleparty.com,index.html,toggle navigation home the issues take action...,[{u' ...
4,americanfreedomunion.com,index.html,home the nationalist times past issues politi...,[{u' ...


In [82]:
def remove_stopwords(text):
    '''
    Multilingual stopwords removal
    '''
    try:
        text = ' '.join([word for word in text.split(' ') if word not in stopwords.words('english')])
        return text
    except:
        return text
    
def textblobsent(text):
    '''
    returns the TextBlob polarity and subjectivity
    '''
    text = text.encode('ascii','replace')
    sent = TextBlob(text).sentiment
    return pd.Series([sent.polarity,sent.subjectivity])

In [112]:
df['CleanText'] = df['text'].apply(remove_stopwords)
sentdf = df['text'].apply(textblobsent)
sentdf.columns = ['Polarity','Subjectivity']
df = pd.concat([df,sentdf],axis=1)

In [113]:
df.columns

Index([u'level_0', u'level_1', u'text', u'entities', u'CleanText',
       u'TextblobPolarity', u'TextblobSubjectivity', u'Polarity',
       u'Subjectivity', u'Polarity', u'Subjectivity'],
      dtype='object')